In [ ]:
# Install kaggle package
!pip install kaggle

# Upload your kaggle.json file (download from kaggle.com/settings)
from google.colab import files
files.upload()  # Upload kaggle.json

# Set up API credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d realalexanderwei/food-com-recipes-with-ingredients-and-tags

# Unzip and load
!unzip food-com-recipes-with-ingredients-and-tags.zip
import pandas as pd
df = pd.read_csv('recipes_ingredients.csv')  # Adjust filename as needed

Dataset URL: https://www.kaggle.com/datasets/realalexanderwei/food-com-recipes-with-ingredients-and-tags
License(s): CC0-1.0
 55% 128M/233M [00:00<00:00, 1.34GB/s]
100% 233M/233M [00:00<00:00, 816MB/s] 
Archive:  food-com-recipes-with-ingredients-and-tags.zip
  inflating: recipes_ingredients.csv  


In [ ]:
#df['cleaned_ingredients'] = df['ingredients_raw'].apply(clean_raw_ingredients)

def strip_bracketsnquotes(ingredients_text):
    ingredients_text = ingredients_text.strip('[]"\'')  # Remove brackets and quotes
    return ingredients_text

def remove_measurements(ingredients_text):
    import re
    ingredients_text = re.sub(r'\d+(\.\d+)?\s*(cups?|tablespoons?|teaspoons?|lbs?|ounces?)', '', ingredients_text)
    return ingredients_text

In [ ]:
#Cleaning Data

columns_to_remove = ['serving_size', 'ingredients']
cleaned_df = df.drop(columns = columns_to_remove)
cleaned_df = cleaned_df.dropna()
cleaned_df['cleaned_ingredients'] = cleaned_df['ingredients_raw'].apply(strip_bracketsnquotes)



In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2", device = "cuda")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:

#should probably adjust fields to be more descriptive, don't hard code weights
def multi_field_embeddings(df, model, weights={'name': 0.2, 'description': 0.3, 'ingredients': 0.4, 'tags': 0.1}):
    """Search across fields separately, then combine scores"""
    import numpy as np

    # Pre-compute embeddings for each field (do this once)
    if not hasattr(df, '_name_embeddings'):
        df._name_embeddings = model.encode(df['name'].tolist())
        df._desc_embeddings = model.encode(df['description'].tolist())
        df._ingr_embeddings = model.encode(df['ingredients_raw'].tolist())
        df._tags_embeddings = model.encode(df['tags'].tolist())

    # Weighted combination
    combined_embeddings = (weights['name'] * df._name_embeddings +
                   weights['description'] * df._desc_embeddings +
                   weights['ingredients'] * df._ingr_embeddings +
                   weights['tags'] * df._tags_embeddings)

    df['recipe_embedding'] = [emb for emb in combined_embeddings]

    return df


In [ ]:
!pip install faiss-cpu --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 112.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.2 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.2 which is incompatible.
opencv-

In [ ]:
import warnings
warnings.filterwarnings('ignore', message='UserWarning')
#!pip install datasets
from datasets import Dataset

df = cleaned_df
sample_size = 10000  # Good for Day 1
df_sample = df.sample(n=sample_size, random_state=42)
multi_field_embeddings(df_sample, model)
# embeddings_dataset = Dataset.from_pandas(df)
# embeddings_dataset.add_faiss_index(column="recipe")

/tmp/ipython-input-1135062346.py:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df._name_embeddings = model.encode(df['name'].tolist())
/tmp/ipython-input-1135062346.py:9: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df._desc_embeddings = model.encode(df['description'].tolist())
/tmp/ipython-input-1135062346.py:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df._ingr_embeddings = model.encode(df['ingredients_raw'].tolist())
/tmp/ipython-input-1135062346.py:11: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df._tags

,id,name,description,ingredients_raw,steps,servings,tags,cleaned_ingredients,recipe_embedding
332210,66221,"Caramel, Peanut and Chocolate Ice Cream",This is a really good flavour combination. Car...,"[""1 1/3 cups superfine sugar"",""10 tables...","[""For caramel: Cook sugar and water in medium ...",1.0,"[""weeknight"", ""time-to-make"", ""course"", ""main-...","1 1/3 cups superfine sugar"",""10 tablespo...","[-0.04881172, -0.047896255, 0.026857592, 0.026..."
122479,508213,Happy Joes Pizza Pasta Knock Off,Pizza pasta,"[""12 ounces shell pasta"",""12 ounces ...","[""Preheat to 350."", ""Cook pasta al dente."", ""C...",12.0,"[""60-minutes-or-less"", ""time-to-make"", ""course...","12 ounces shell pasta"",""12 ounces pi...","[-0.08462217, -0.014965706, 0.016254146, 0.028..."
21955,475959,Indian Steamed Mussels,This recipe is from some of my partying friend...,"[""1 tablespoon unsalted butter"",""1 tabl...","[""Heat butter and oil in a 12″ skillet over me...",2.0,"[""60-minutes-or-less"", ""time-to-make"", ""course...","1 tablespoon unsalted butter"",""1 tables...","[-0.05836663, -0.016064651, -0.0059680548, -0...."
49451,456912,Zucchinis (Courgettes) Stuffed With Couscous,From the Italian Taste http://www.theitalian...,"[""6 round courgettes"",""150 g couscou...","[""Trim the upper end and wash courgettes very ...",3.0,"[""time-to-make"", ""course"", ""main-ingredient"", ...","6 round courgettes"",""150 g couscous ...","[-0.033675224, -0.02196356, -0.032266226, 0.00..."
187944,477875,Raw Lemon Cookies/Macaroons,"I go tthis off of a web site, and as usual put...","[""2 cups almonds, soaked for 4 hours "",""2...","[""Process almonds in food processor until they...",1.0,"[""course"", ""preparation"", ""drop-cookies"", ""des...","2 cups almonds, soaked for 4 hours "",""2 ...","[-0.12557244, -0.034176517, 0.006822992, 0.004..."
...,...,...,...,...,...,...,...,...,...
224985,371264,Dianthum,I got the idea when a Sunday brunch place we g...,"[""1 large watermelon"",""1 (750 ml) bottl...","[""Refrigerate watermelon and champagne of your...",12.0,"[""lactose"", ""course"", ""main-ingredient"", ""prep...","1 large watermelon"",""1 (750 ml) bottle ...","[-0.07728411, 0.033218995, -0.011249106, 0.000..."
65100,424025,Shrimp and Melon Gazpacho,A different version of gazpacho and a deliciou...,"[""1 pint vegetable broth"",""1 -1 1/2 lb ...","[""First make a court bouillon-poaching liquid ...",6.0,"[""weeknight"", ""60-minutes-or-less"", ""time-to-m...","1 pint vegetable broth"",""1 -1 1/2 lb ...","[-0.027397845, 0.014961951, -0.047963504, 0.02..."
378080,476895,Broiled Citrus &amp; Cranberries,"From Whole Living Nov 2011""After a rich, filli...","[""1 tangerine (peeled \u0026 segmented)""...","[""Heat broiler; divide fruit among four shallo...",4.0,"[""30-minutes-or-less"", ""time-to-make"", ""course...","1 tangerine (peeled \u0026 segmented)"",""...","[-0.06260746, -0.036351018, -0.010186632, 0.02..."
422584,362801,Shabbat Chicken and Rice All-In-One,You can vary this by including different veget...,"[""1 chicken, cut up and skin removed "",""...","[""Pour rice into the bottom of a 9x13"" glass b...",4.0,"[""time-to-make"", ""course"", ""main-ingredient"", ...","1 chicken, cut up and skin removed "",""1 ...","[0.004936608, -0.035276372, -0.020226311, 0.01..."


In [ ]:
print(df_sample.shape)

(10000, 9)


In [ ]:
import faiss
import numpy as np

def setup_recipe_search(df_sample, model):
    """Set up recipe search with pure FAISS"""

    # Get embeddings from your DataFrame5
    embeddings_list = df_sample['recipe_embedding'].tolist()
    embeddings_array = np.array(embeddings_list, dtype='float32')

    print(f"Embeddings shape: {embeddings_array.shape}")

    # Create FAISS index
    index = faiss.IndexFlatIP(embeddings_array.shape[1])
    index.add(embeddings_array)

    print(f"FAISS index created with {index.ntotal} recipes")
    return index, embeddings_array

def search_recipes(df_sample, model, index, query, k=5):
    """Search for similar recipes"""

    # Get query embedding
    query_embedding = model.encode([query]).astype('float32')

    # Search
    scores, indices = index.search(query_embedding, k)

    # Get results
    results = df_sample.iloc[indices[0]].copy()
    results['scores'] = scores[0]
    results = results.sort_values('scores', ascending=False)

    return results

# Usage:
# 1. Create embeddings (your existing function)
df_sample = multi_field_embeddings(df_sample, model)

# 2. Set up FAISS index (skip HuggingFace entirely)
index, embeddings_array = setup_recipe_search(df_sample, model)

# 3. Search
recipe_query = "Butter Chicken with Greens and Rice"
results = search_recipes(df_sample, model, index, recipe_query, k=5)

# 4. Display results
print("Top 5 Similar Recipes:")
print("=" * 50)
for _, row in results.iterrows():
    print(f"Name: {row['name']}")
    print(f"Description: {row['description']}")
    print(f"Tags: {row['tags']}")
    print(f"Score: {row['scores']:.4f}")
    print("=" * 50)
    print()

Embeddings shape: (10000, 384)
FAISS index created with 10000 recipes
Top 5 Similar Recipes:
Name: Chicken and Rice
Description: If you like chicken and rice dishes this recipe will rate high on your list.
Tags: ["weeknight", "60-minutes-or-less", "time-to-make", "course", "main-ingredient", "cuisine", "preparation", "occasion", "north-american", "main-dish", "poultry", "rice", "easy", "european", "dinner-party", "chicken", "dietary", "low-sodium", "low-carb", "low-in-something", "meat", "pasta-rice-and-grains", "3-steps-or-less"]
Score: 0.6353

Name: Chicken and Broccoli Fried Brown Rice
Description: I've been experimenting with Chinese food lately and have been wanting to make fried rice.  Considering this was my first time making it, it turned out pretty good!
Tags: ["weeknight", "30-minutes-or-less", "time-to-make", "course", "main-ingredient", "cuisine", "preparation", "occasion", "main-dish", "rice", "asian", "chinese", "dietary", "one-dish-meal", "low-saturated-fat", "low-calori